## Community detection for all metropolitan areas

In [116]:
import pandas as pd
import numpy as np
import geopandas as gpd
import seaborn as sns
import networkx as nx

import scipy

import matplotlib.cm as cm
import matplotlib.pyplot as plt
import json
import community as community_louvain
from copy import deepcopy
# from modularity_maximization.utils import get_modularity

from itertools import product

import networkx.algorithms.community as nx_comm

%matplotlib inline

In [98]:
# ez tolti be az octave cell magicet
%load_ext oct2py.ipython

The oct2py.ipython extension is already loaded. To reload it, use:
  %reload_ext oct2py.ipython


In [10]:
from oct2py import octave
# octave.addpath('/ModularitySpaGN.m')

OSError: octave-cli not found, please see README

In [5]:
import oct2py

ModuleNotFoundError: No module named 'oct2py'

### data

In [99]:
# data IN
mobility = pd.read_csv("../data/usageousers_city_mobility_CT_networks.rpt.gz") ## A MOBILITY HÁLÓBÓL SZÁMOLOK POSITION-T, AZÉRT KELL!

follow_hh = pd.read_csv("../data/usageousers_city_follower_CT_HH_networks.rpt.gz")
follow_hh = follow_hh.rename(columns={"tract_home.1": "tract_home_1"})

cbsacode = pd.read_csv("../data/cbsacode_shortname_tracts.csv",sep=";", index_col=0)

In [106]:
tract_sum = cbsacode.groupby('cbsacode')[['cbsacode']].count() # number of tracts per city
tract_sum = tract_sum.rename(columns={'cbsacode': 'sum_tracts'})

In [102]:
# unique values in the network counted by side of edge

mob_u_h = mobility.tract_home.tolist()
mob_u_w = mobility.tract_work.tolist()
u_geoid_mob = set([*mob_u_h,*mob_u_w])
cbsacode['in_mob'] = cbsacode.geoid.isin(u_geoid_mob).astype(int)

fol_u_h = follow_hh.tract_home.tolist()
fol_u_h1 = follow_hh.tract_home_1.tolist()
u_geoid_fol = set([*fol_u_h,*fol_u_h1])
cbsacode['in_fol'] = cbsacode.geoid.isin(u_geoid_fol).astype(int)

In [113]:
netw_count = cbsacode.groupby('cbsacode')[['in_mob','in_fol']].sum()

In [114]:
city_df = pd.merge(tract_sum, netw_count, left_index=True, right_index=True)

In [ ]:
nx_comm.modularity()

G = nx.barbell_graph(3, 0)

nx_comm.modularity(G, [{0, 1, 2}, {3, 4, 5}])
0.35714285714285715

nx_comm.modularity(G, nx_comm.label_propagation_communities(G))
0.35714285714285715


In [115]:
city_df

,sum_tracts,in_mob,in_fol
cbsacode,,,
12060.0,947,947,940
12420.0,348,348,344
12580.0,674,674,666
13820.0,263,263,261
14460.0,990,990,983
15380.0,295,295,292
16740.0,536,536,528
16980.0,2201,2201,2176
17140.0,495,495,492


In [ ]:
mobility.groupby('cbsacode')

In [31]:
mobility.groupby('cbsacode')[['tract_home']].nunique()

,tract_home
cbsacode,
12060,947
12420,349
12580,674
13820,263
14460,990
15380,295
16740,536
16980,2201
17140,495


In [12]:
tract = mobility.groupby('tract_home')[['cnt']].sum()

In [13]:
tract

,cnt
tract_home,
14000US01007010001,9
14000US01007010002,14
14000US01007010003,10
14000US01007010004,22
14000US01009050101,21
14000US01009050102,29
14000US01009050200,20
14000US01009050300,12
14000US01009050400,1


In [ ]:
follow_hh = follow_hh.merge(tract_outdeg_mob, left_on='tract_home', right_index=True)

N = np.matrix([tract_outdeg_mob.loc[k].iloc[0] for k in g_fol_hh.nodes()]).T
# modify N